## Points to remember

Write down the points to be remembered in this cell while coding, so it might be helpful to others working on same code - different functions

1) Every current node (object) will be updated as visited after it goes through inference rules function <br>
2) send 3 arguments to init()<br>
3) import this file to the main file<br>
4) declaring only one set of variables for: agent_matrix,helper_matrix, agent_gridworld, full_gridworld in the main file. This file is only to implement functionalities of inference engine<br>

In [12]:
import numpy as np

In [5]:
class partial_sensing:

#         self.Nx = Nx   the number of neighbors cell x has.
#         self.visited = visited  Whether or not cell x has been visited.
#         self.blocked = blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed.
#         self.Cx = Cx   the number of neighbors of x that are sensed to be blocked.
#         self.Bx = Bx   the number of neighbors of x that have been confirmed to be blocked..
#         self.Ex = Ex   the number of neighbors of x that have been confirmed to be empty.
#         self.Hx = Hx  the number of neighbors of x that are still hidden or unconfirmed either way.

    def __init__(self, row, col, maze_size):
        self.row = row
        self.column = col
        self.neighbor_list = [] #need to find a node's neighbor indices and append to this list
        self.Nx = 0
        self.Cx = 0
        self.Bx = 0
        self.Ex = 0
        self.Hx = 0
        self.visited = False
        self.blocked = 0
        self.free_neighbors = []
        self.blocked_neighbors = []

    def find_children(self):
        """
        Input:The current node
        Output: number of children
        """
        self.Nx = len(children_list)

    def Check_visited(self):
        """
        Input:The current node
        Output: Mark true if visited else false
        """
        if Node in visited_list:
            self.visited = True
        else:
            self.visited = False

    def check_blocked(self):
        """
        Input:The current node
        Output: Mark true if blocked else false
        """
        if Node == "blocked":
            self.blocked = True
        else:
            self.blocked = False

    def number_of_blocked_neighbors_sensed(self, helper_matrix):
        """
        Input:The current node
        Output: mark the number of neighbours that are blocked
        """
        self.Cx = helper_matrix[self.row][self.column]  # the number of neighbors of x that are sensed to be blocked.

#     def number_of_blocked_neighbors(self):
#         """
#         Input:The current node
#         Output: mark the number of neighbours that are empty
#         """
#         self.Ex = self.Nx - self.Cx # the number of neighbors of x that have been confirmed to be empty.

    def number_of_unconfirmed_neighbors(self, visited_list):
        """
        Input:The current node
        Output: mark the number of neighbours that are yet to be visited/unconfirmed
        """
        neighbor_list = self.neighbor_list
        # compare neighbors list with visited list to check the visited neighbors
        neighbors_visited = list(set(neighbor_list).intersection(visited_list))

        self.Hx = self.Nx - len(neighbors_visited) # the number of neighbors of x that are still hidden or unconfirmed either way

    def display_node(self):
        print("Node: ", self.row, "," , self.column)
        print("\n the number of neighbors cell x has: ", self.Nx)
        print("visited  Whether or not cell x has been visited: ", self.visited)
        print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
        print("the number of neighbors of x that are sensed to be blocked: ", self.Cx)
        print("the number of neighbors of x that have been confirmed to be blocked: ", self.Bx)
        print("the number of neighbors of x that have been confirmed to be empty: ", self.Ex)
        print("the number of neighbors of x that are still hidden or unconfirmed either way: ", self.Hx)
        print("The undiscovered neighbors are: ", self.neighbor_list)
        print("The confirmed neighbors that are free: ", self.free_neighbors)
        print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

    def inference_engine(self, visited_list, knowledge_base):
        row = self.row
        col = self.column
        neighbors_visited = list(set(self.neighbor_list).intersection(visited_list))

        # if sensed blocked nodes = 0, the number of confirmed blocked nodes will always remain zero
        if self.Cx == 0:
            self.Bx = 0

        # If Hx = 0: nothing remains to be inferred about cell x.
        if self.Hx == 0:
            print("nothing remains to be inferred about cell", self.row, ",",  self.column )

        if self.Nx == self.Ex:
            self.Bx == 0
            for node in self.neighbor_list:
                node_row = node[0]
                node_col = node[1]
                node_object = knowledge_base[node_row][node_col]
                node_object.blocked = 0
                if node not in visited_list:
                    print("Discovered that ", node ," is free node")

        if self.Nx == self.Bx:
            self.Ex == 0
            for node in self.neighbor_list:
                node_row = node[0]
                node_col = node[1]
                node_object = knowledge_base[node_row][node_col]
                node_object.blocked = 1
                if node not in visited_list:
                    print("Discovered that ", node ," is blocked node")

        # if self is blocked, then updated all the neighboring visited nodes with Bx += 1
        if self.blocked == 1:
            for neighbor_node in self.neighbor_list:
                neighbor_row = neighbor_node[0]
                neighbor_col = neighbor_node[1]
                if (neighbor_row, neighbor_col) in visited_list:
                    neighbor_node_object = knowledge_base[neighbor_row][neighbor_col]
                    if (neighbor_row, neighbor_col) in visited_list:
                        self.Hx -= 1
                    neighbor_node_object.Bx += 1
                    if neighbor_node_object.Hx >=0:
                        neighbor_node_object.Hx -= 1
                    #appending blocked node to neighbor's blocked list
                    if (row,col) not in neighbor_node_object.blocked_neighbors:
                        neighbor_node_object.blocked_neighbors.append((row,col))
    #                         if (row,col) in neighbor_node_object.neighbor_list:
                        if (row,col) in neighbor_node_object.neighbor_list:
                            neighbor_node_object.neighbor_list.remove((row,col))
        else:
            for neighbor_node in self.neighbor_list:
                neighbor_row = neighbor_node[0]
                neighbor_col = neighbor_node[1]
                if (neighbor_row, neighbor_col) in visited_list:
                    neighbor_node_object = knowledge_base[neighbor_row][neighbor_col]

                    #appending blocked node to neighbor's free list
                    if ((row,col)) not in neighbor_node_object.free_neighbors:
                        neighbor_node_object.free_neighbors.append((row,col))
#                         if (row,col) in neighbor_node_object.neighbor_list:
                        if (row,col) in neighbor_node_object.neighbor_list:
                            neighbor_node_object.neighbor_list.remove((row,col))
                    if (neighbor_row, neighbor_col) in visited_list:
                        self.Hx -= 1
                    if neighbor_node_object.Hx >=0:
                        neighbor_node_object.Hx -= 1

#         for node in visited_list:
#             visited_row = node[0]
#             visited_col = node[1]
#             neighbor_node_object = knowledge_base[visited_row][visited_col]
#             if (visited_row, visited_col) in self.neighbor_list:
#                 if neighbor_node_object.blocked == 0:
#                     self.free_neighbors.append((visited_row, visited_col))
#                     self.neighbor_list.remove((visited_row, visited_col))
#                 if neighbor_node_object.blocked == 1:
#                     self.blocked_neighbors.append((visited_row, visited_col))
#                     if 
#                     self.neighbor_list.remove((visited_row, visited_col))